In [ ]:
# Extracting a full list of Cryptocurrencies

# Cryptocurrency Time-Series for N-CryptoAsset Portfolio Analysis in Python
# http://www.quantatrisk.com/2017/03/20/download-crypto-currency-time-series-portfolio-python/
# With  CryptoComapre.com API  https://www.cryptocompare.com/

import json
from pprint import pprint 
from bs4 import BeautifulSoup
import requests 
 
url = "https://www.cryptocompare.com/api/data/coinlist/"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
data = json.loads(soup.prettify()) # type(data) -> dict
data = data['Data']  # choose key = 'Data'
pprint(data)  # display the content


# check coin list
# crypto_lst = sorted(list(data.keys()))
# print(crypto_lst) 

In [ ]:
# Getting a List of Cryptocurrencies sorted by their most current Market Cap

import pandas as pd
import json
from bs4 import BeautifulSoup
from pprint import pprint 
import requests
 
url = "https://api.coinmarketcap.com/v1/ticker/"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
data = json.loads(soup.prettify())  #list

# create an empty DataFrame
df = pd.DataFrame(columns=["Ticker", "MarketCap"]) 

for i in range(len(data)):
    df.loc[len(df)] = [data[i]['symbol'], data[i]['market_cap_usd']]

# order by marketcap
df.sort_values(by=['MarketCap'])

# apply conversion to numeric as 'df' contains lots of 'None' string as values
df.MarketCap = pd.to_numeric(df.MarketCap)    


# marketcap >20e6
P = df[df.MarketCap > 20e6]
print(P, end="\n\n")

# print their name 
portfolio = list(P.Ticker) #這滿酷,直接將一個column變list
print(portfolio)

# df.head()

In [2]:
# Downloading daily price-series OHLC for Cryptocurrencies
# OHLC Open-high-low-close chart

import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import json
from bs4 import BeautifulSoup
import requests
%pylab inline
from datetime import datetime

def timestamp2date(timestamp):
    # function converts a Unix timestamp into Gregorian date
    return datetime.fromtimestamp(int(timestamp)).strftime('%Y-%m-%d')

def date2timestamp(date):
    # function coverts Gregorian date in a given format to timestamp
    return datetime.strptime(date_today, '%Y-%m-%d').timestamp()

def fetchCryptoOHLC(fsym, tsym):
    # function fetches a crypto price-series for fsym/tsym and stores
    # it in pandas DataFrame
 
    cols = ['date', 'timestamp', 'open', 'high', 'low', 'close']
    lst = ['time', 'open', 'high', 'low', 'close']
 
    timestamp_today = datetime.today().timestamp()
    curr_timestamp = timestamp_today
 
    for j in range(2):
        df = pd.DataFrame(columns=cols)  # empty df
        url = "https://min-api.cryptocompare.com/data/histoday?fsym=" + fsym + "&tsym=" + tsym + "&toTs=" + str(int(curr_timestamp)) + "&limit=2000"
        response = requests.get(url)
        print(j, url)
        soup = BeautifulSoup(response.content, "html.parser")
        dic = json.loads(soup.prettify())
        for i in range(1, 2001):
            tmp = []
            for e in enumerate(lst):
                x = e[0]
                y = dic['Data'][i][e[1]]
                if(x == 0):
                    tmp.append(str(timestamp2date(y)))
                tmp.append(y)
            if(np.sum(tmp[-4::]) > 0):
                df.loc[len(df)] = np.array(tmp)
        df.index = pd.to_datetime(df.date)
        df.drop('date', axis=1, inplace=True)
        curr_timestamp = int(df.ix[0][0])
        if(j == 0):
            df0 = df.copy()
        else:
            data = pd.concat([df, df0], axis=0)
 
    return data

Populating the interactive namespace from numpy and matplotlib


In [3]:
fsym = "BTC"
tsym = "USD"
data = fetchCryptoOHLC(fsym, tsym)

# print the BTC/USD OHLC price-series
print(data)


0 https://min-api.cryptocompare.com/data/histoday?fsym=BTC&tsym=USD&toTs=1501836930&limit=2000
1 https://min-api.cryptocompare.com/data/histoday?fsym=BTC&tsym=USD&toTs=1329091200&limit=2000
             timestamp     open     high      low    close
date                                                      
2010-07-17  1279324800  0.04951  0.04951  0.04951  0.04951
2010-07-18  1279411200  0.04951  0.08585  0.05941  0.08584
2010-07-19  1279497600  0.08584  0.09307  0.07723   0.0808
2010-07-20  1279584000   0.0808  0.08181  0.07426  0.07474
2010-07-21  1279670400  0.07474  0.07921  0.06634  0.07921
2010-07-22  1279756800  0.07921  0.08181   0.0505   0.0505
2010-07-23  1279843200   0.0505  0.06767   0.0505  0.06262
2010-07-24  1279929600  0.06262  0.06161  0.05049  0.05454
2010-07-25  1280016000  0.05454  0.05941   0.0505   0.0505
2010-07-26  1280102400   0.0505    0.056     0.05    0.056
2010-07-27  1280188800    0.056   0.0605    0.053     0.06
2010-07-28  1280275200     0.06    0.062   

In [4]:
# plot them all
# plt.figure(figsize=(10,4))
# plt.plot(data.open)
# plt.plot(data.high)
# plt.plot(data.low)
# plt.plot(data.close)
# plt.legend(loc=2)
# plt.title(fsym, fontsize=12)
# plt.ylabel(tsym, fontsize=12)

In [11]:
# bokeh  待繼續研究

from bokeh.plotting import figure, output_notebook, show
from bokeh.charts import TimeSeries, output_file, show, Line
from numpy import cos, linspace

p = figure(width=500, height=500)
p.line(source=data, x='date', y='open')
show(p)

D:\Anaconda3\lib\site-packages\bokeh\util\deprecation.py:34: BokehDeprecationWarning:


The bokeh.charts API has moved to a separate 'bkcharts' package.

This compatibility shim will remain until Bokeh 1.0 is released.
After that, if you want to use this API you will have to install
the bkcharts package explicitly.




In [15]:
# plotly 
# https://plot.ly/python/line-charts/#line-plot-modes
# https://plot.ly/python/time-series/
import plotly.plotly as py
import plotly.graph_objs as go

plotly.tools.set_credentials_file(username='KBJIYU', api_key='QOUekjVYn9nlK2g1Nm6W')

open_price = go.Scatter(
    x = list(data.index),
    y = list(data.open),
    mode = 'lines',
    name = 'open_price'
)
close_price = go.Scatter(
    x = list(data.index),
    y = list(data.close),
    mode = 'lines',
    name = 'close_price'
)
high_price = go.Scatter(
    x = list(data.index),
    y = list(data.high),
    mode = 'lines',
    name = 'high_price'
)
low_price = go.Scatter(
    x = list(data.index),
    y = list(data.low),
    mode = 'lines',
    name = 'low_price'
)

graph_data = [open_price, close_price, high_price, low_price]

# Online Plot
# py.iplot(graph_data, filename = 'line-mode')

# Offline Plot 
plotly.offline.plot({
    "data": graph_data
    #"layout": Layout(title="BTC_DAILY_PRICE")
})

'file://C:\\Users\\kbjiyu-NB\\Documents\\workingspacePython\\python scraper\\bitcoin新聞_jupter_code\\temp-plot.html'

In [ ]:
# https://plot.ly/python/ohlc-charts/
# try this 



In [ ]:
# and try scrape historic minutes data